In [12]:
import pandas as pd
import tensorflow as tf
import math
import tensorflow_hub as hub
import tensorflow_text as text
import pickle
import os

In [2]:
sentence_encoder = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2023-03-01 19:28:27.902767: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-03-01 19:28:27.945015: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 19:28:27.945503: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 computeCapability: 8.6
coreClock: 1.71GHz coreCount: 68 deviceMemorySize: 9.78GiB deviceMemoryBandwidth: 707.88GiB/s
2023-03-01 19:28:27.945547: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-01 19:28:27.945912: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 000

In [3]:
def get_sentence_score(sent1, sent2):
    sts_encode1 = tf.nn.l2_normalize(sentence_encoder(sent1), axis=1)
    sts_encode2 = tf.nn.l2_normalize(sentence_encoder(sent2), axis=1)
    cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
    clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
    scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
    average = tf.math.reduce_mean(scores)
    return tf.get_static_value(average)

In [7]:
def score_dataset(watermarked, unmarked):     
    if len(watermarked) != len(unmarked):
        print("Size mismatch.")
        return
    
    score_total = 0
    
    for i in range(len(watermarked)):
        score_total += get_sentence_score(watermarked[i], unmarked[i])
        
    return score_total / len(watermarked)

In [8]:
marked_data_folder = 'AllWord2VecV1Watermarked/'
unmarked_data_folder = 'AllWord2VecV1Unmarked/'

with open(unmarked_data_folder + 'unmarked.pkl', 'rb') as file:
    read_unmarked = pickle.load(file)
    
with open(marked_data_folder + 'watermarked.pkl', 'rb') as file:
    read_watermarked = pickle.load(file)

In [9]:
score_dataset(read_watermarked, read_unmarked)

0.9856100383722005

In [14]:
direct_marked_data_folder = 'AllDirectWatermarkedV2/'
direct_watermarked = []

for file in sorted(os.listdir(direct_marked_data_folder)):
    fpath = os.path.join(direct_marked_data_folder, file)
    
    if os.path.isfile(fpath) and fpath.lower().endswith(".pkl"):
        with open(fpath, 'rb') as file:
            data = pickle.load(file)
        
        direct_watermarked += data

In [15]:
direct_unmarked_data_folder = 'AllDirectUnmarkedV2/'
direct_unmarked = []

for file in sorted(os.listdir(direct_unmarked_data_folder)):
    fpath = os.path.join(direct_unmarked_data_folder, file)
    
    if os.path.isfile(fpath) and fpath.lower().endswith(".pkl"):
        with open(fpath, 'rb') as file:
            data = pickle.load(file)
        
        direct_unmarked += data

## Direct Watermarked BERT sentence similarity

In [16]:
score_dataset(direct_watermarked, direct_unmarked)

0.9794132883378633

## Word2Vec watermarked BERT sentence similiarity

In [17]:
score_dataset(read_watermarked[:len(direct_watermarked)], read_unmarked[:len(direct_unmarked)])

0.9885337761205688

In [18]:
len(direct_watermarked)

1738